In [1]:
from keras import applications
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.layers.normalization import BatchNormalization
from keras import regularizers
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.utils import layer_utils, np_utils
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.datasets import make_classification
from sklearn.preprocessing import label_binarize
from scipy import interp
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from hyperas import optim
from hyperopt import Trials, STATUS_OK, tpe
from hyperas.distributions import choice, uniform, conditional
from hyperas.utils import eval_hyperopt_space
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

Using TensorFlow backend.


In [2]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [3]:
def data():
    train_dir = "D:/retinal_samples/basian_opt_testing/new_new_train/augmented_224/train"
    validation_dir = "D:/retinal_samples/basian_opt_testing/new_valid"
    
    image_width = 224
    image_height = 224
    batchsize = 16
    
    train_datagen = ImageDataGenerator(rescale=1./255)
    validation_datagen = ImageDataGenerator(rescale=1./255)
    
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_width, image_height),
        batch_size=batchsize,
        shuffle=True,
        class_mode='categorical')

    validation_generator = validation_datagen.flow_from_directory(
            validation_dir,
            target_size=(image_width, image_height),
            batch_size=batchsize,
            class_mode='categorical',
            shuffle=False)
    
    return train_generator, validation_generator

In [4]:
def model(train_generator, validation_generator):
    vgg_16_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(image_width, image_height, 3))
    model_check_point_loc = "D:/retinal_samples/basian_opt_testing/models/vgg16/vgg16_dr_model_check.h5"
    
    for layer in vgg_16_model.layers[:]:
        layer.trainable = False
    
    model = models.Sequential()
    model.add(vgg_16_model)
    model.add(layers.Flatten())
    model.add(Dense({{choice([256, 512, 1024])}}, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout({{uniform(0.4, 1)}}))
    
    choice_val = {{choice(['one', 'two'])}}
    if choice_val == 'two':
        model.add(Dense({{choice([128, 256, 512])}}, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout({{uniform(0.4, 1)}}))
                  
    model.add(Dense(5, activation='softmax', kernel_regularizer=regularizers.l2({{choice([0.0001, 0.001])}})))
              
    lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.2, cooldown=0, patience=10, min_lr=1e-5)
    early_stopper = EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=20)
    model_checkpoint = ModelCheckpoint(model_check_point_loc, monitor='val_acc', save_best_only=True, save_weights_only=False, mode='auto')
    
    callbacks = [lr_reducer, early_stopper, model_checkpoint]
              
    selected_optimizer = {{choice(['rmsprop', 'adam', 'sgd'])}}
    if selected_optimizer == 'adam':
        model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr={{choice([1e-3, 1e-4])}}),
              metrics=['acc'])
    elif selected_optimizer == 'rmsprop':
        model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr={{choice([1e-3, 1e-4])}}),
              metrics=['acc'])
    else:
        model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr={{choice([1e-3, 1e-4])}}, momentum=0.9),
              metrics=['acc'])
    history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples//train_generator.batch_size ,
      epochs={{choice([25, 50, 75, 100])}},
      callbacks=callbacks,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples//validation_generator.batch_size,
      verbose=1)
    score, acc = model.evaluate_generator(generator=validation_generator, 
                                      steps=validation_generator.samples // validation_generator.batch_size)
    
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [5]:
train_generator, validation_generator = data()
best_run, best_model, space = optim.minimize(model=model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          eval_space=True,
                                          return_space=True,
                                          notebook_name='vgg16_model_basian_opt')

best_model.save('D:/retinal_samples/basian_opt_testing/models/vgg16/vgg16_dr_basian_opt.h5')

Found 2000 images belonging to 5 classes.
Found 55 images belonging to 5 classes.
>>> Imports:
#coding=utf-8

try:
    from keras import applications
except:
    pass

try:
    from keras import models
except:
    pass

try:
    from keras import layers
except:
    pass

try:
    from keras import optimizers
except:
    pass

try:
    from keras.preprocessing.image import ImageDataGenerator
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers import Dropout, Flatten, Dense
except:
    pass

try:
    from keras.layers.normalization import BatchNormalization
except:
    pass

try:
    from keras import regularizers
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    from keras.models import load_model
except:
    pass

try:
    from keras.utils import layer_utils, np_utils
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from sklearn.metrics import classification_report, confu

125/125 [==============================] - 127s 1s/step - loss: 1.2855 - acc: 0.4970 - val_loss: 1.3638 - val_acc: 0.4167
Epoch 14/50
125/125 [==============================] - 127s 1s/step - loss: 1.2268 - acc: 0.5100 - val_loss: 1.3999 - val_acc: 0.3333
Epoch 15/50
125/125 [==============================] - 127s 1s/step - loss: 1.2379 - acc: 0.5045 - val_loss: 1.4648 - val_acc: 0.4375
Epoch 16/50
125/125 [==============================] - 127s 1s/step - loss: 1.2409 - acc: 0.5290 - val_loss: 1.5005 - val_acc: 0.4167
Epoch 17/50
125/125 [==============================] - 127s 1s/step - loss: 1.1659 - acc: 0.5235 - val_loss: 2.0675 - val_acc: 0.2917
Epoch 18/50
125/125 [==============================] - 127s 1s/step - loss: 1.1451 - acc: 0.5500 - val_loss: 1.3610 - val_acc: 0.4583
Epoch 19/50
125/125 [==============================] - 127s 1s/step - loss: 1.0786 - acc: 0.5420 - val_loss: 1.2775 - val_acc: 0.5208
Epoch 20/50
125/125 [==============================] - 126s 1s/step - loss

125/125 [==============================] - 128s 1s/step - loss: 1.4326 - acc: 0.3840 - val_loss: 1.3995 - val_acc: 0.3542
Epoch 36/75
125/125 [==============================] - 127s 1s/step - loss: 1.4233 - acc: 0.3915 - val_loss: 1.3607 - val_acc: 0.4583
Epoch 37/75
125/125 [==============================] - 128s 1s/step - loss: 1.4215 - acc: 0.3750 - val_loss: 1.3744 - val_acc: 0.3958
Epoch 38/75
125/125 [==============================] - 127s 1s/step - loss: 1.4383 - acc: 0.3810 - val_loss: 1.3802 - val_acc: 0.3958
Epoch 39/75
125/125 [==============================] - 128s 1s/step - loss: 1.4008 - acc: 0.3780 - val_loss: 1.3867 - val_acc: 0.3542
Epoch 40/75
125/125 [==============================] - 127s 1s/step - loss: 1.4391 - acc: 0.3725 - val_loss: 1.3787 - val_acc: 0.3750
Epoch 41/75
125/125 [==============================] - 128s 1s/step - loss: 1.4249 - acc: 0.3790 - val_loss: 1.3787 - val_acc: 0.3750
Epoch 42/75
125/125 [==============================] - 127s 1s/step - loss

KeyboardInterrupt: 

In [ ]:
print("Evaluation of best performing model:")
print(best_model.evaluate(validation_generator))
print("Best performing model chosen hyper-parameters:")
print(best_run)
print("Real hyper-parameters:")
real_param_values = eval_hyperopt_space(space, best_run)
print(real_param_values)

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()